<a href="https://colab.research.google.com/github/umut7/Customer-Loyalty-Prediction/blob/main/MarketingAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
dataset = pd.read_excel("a1_Dataset_10Percent.xlsx")

In [3]:
dataset.shape()

(22223, 11)

In [4]:
dataset.head()

,ID,DemAffl,DemAge,DemClusterGroup,DemGender,DemReg,DemTVReg,LoyalClass,LoyalSpend,LoyalTime,TargetBuy
0,17147654,5.0,NaN,NaN,NaN,NaN,NaN,Tin,0.01,5.0,0
1,8415498,15.0,NaN,NaN,M,NaN,NaN,Gold,8000.00,5.0,1
2,12107603,NaN,NaN,NaN,M,Midlands,East,Tin,0.01,NaN,1
3,14400995,8.0,28.0,NaN,F,NaN,NaN,Tin,0.01,NaN,1
4,28724674,14.0,67.0,NaN,NaN,NaN,NaN,Tin,0.01,7.0,0


In [5]:
dataset = dataset.drop(['ID'], axis=1)
dataset.head()

,DemAffl,DemAge,DemClusterGroup,DemGender,DemReg,DemTVReg,LoyalClass,LoyalSpend,LoyalTime,TargetBuy
0,5.0,NaN,NaN,NaN,NaN,NaN,Tin,0.01,5.0,0
1,15.0,NaN,NaN,M,NaN,NaN,Gold,8000.00,5.0,1
2,NaN,NaN,NaN,M,Midlands,East,Tin,0.01,NaN,1
3,8.0,28.0,NaN,F,NaN,NaN,Tin,0.01,NaN,1
4,14.0,67.0,NaN,NaN,NaN,NaN,Tin,0.01,7.0,0


In [6]:
dataset.isna().sum()

,0
DemAffl,1085
DemAge,1508
DemClusterGroup,674
DemGender,2512
DemReg,465
DemTVReg,465
LoyalClass,0
LoyalSpend,0
LoyalTime,281
TargetBuy,0


In [7]:
dataset['DemAffl']=dataset['DemAffl'].fillna(dataset['DemAffl'].mode()[0])
dataset['DemAge']=dataset['DemAge'].fillna(dataset['DemAge'].mode()[0])
dataset['DemClusterGroup']=dataset['DemClusterGroup'].fillna(dataset['DemClusterGroup'].mode()[0])
dataset['DemGender']=dataset['DemGender'].fillna(dataset['DemGender'].mode()[0])
dataset['DemReg']=dataset['DemReg'].fillna(dataset['DemReg'].mode()[0])
dataset['DemTVReg']=dataset['DemTVReg'].fillna(dataset['DemTVReg'].mode()[0])
dataset['LoyalTime']=dataset['LoyalTime'].fillna(dataset['LoyalTime'].mode()[0])

In [8]:
dataset.isna().sum()

,0
DemAffl,0
DemAge,0
DemClusterGroup,0
DemGender,0
DemReg,0
DemTVReg,0
LoyalClass,0
LoyalSpend,0
LoyalTime,0
TargetBuy,0


In [9]:
dataset.head()

,DemAffl,DemAge,DemClusterGroup,DemGender,DemReg,DemTVReg,LoyalClass,LoyalSpend,LoyalTime,TargetBuy
0,5.0,51.0,C,F,South East,London,Tin,0.01,5.0,0
1,15.0,51.0,C,M,South East,London,Gold,8000.00,5.0,1
2,8.0,51.0,C,M,Midlands,East,Tin,0.01,5.0,1
3,8.0,28.0,C,F,South East,London,Tin,0.01,5.0,1
4,14.0,67.0,C,F,South East,London,Tin,0.01,7.0,0


In [10]:
from re import L
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()
dataset['DemClusterGroup'] = number.fit_transform(dataset['DemClusterGroup'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'U': 6}


In [11]:
dataset['DemGender'] = number.fit_transform(dataset['DemGender'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

dataset['DemReg'] = number.fit_transform(dataset['DemReg'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

dataset['DemTVReg'] = number.fit_transform(dataset['DemTVReg'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

dataset['LoyalClass'] = number.fit_transform(dataset['LoyalClass'].astype('str'))
integer_mapping = {l: i for i, l in enumerate(number.classes_)}
print(integer_mapping)

{'F': 0, 'M': 1, 'U': 2}
{'Midlands': 0, 'North': 1, 'Scottish': 2, 'South East': 3, 'South West': 4}
{'Border': 0, 'C Scotland': 1, 'East': 2, 'London': 3, 'Midlands': 4, 'N East': 5, 'N Scot': 6, 'N West': 7, 'S & S East': 8, 'S West': 9, 'Ulster': 10, 'Wales & West': 11, 'Yorkshire': 12}
{'Gold': 0, 'Platinum': 1, 'Silver': 2, 'Tin': 3}


In [12]:
dataset.head()

,DemAffl,DemAge,DemClusterGroup,DemGender,DemReg,DemTVReg,LoyalClass,LoyalSpend,LoyalTime,TargetBuy
0,5.0,51.0,2,0,3,3,3,0.01,5.0,0
1,15.0,51.0,2,1,3,3,0,8000.00,5.0,1
2,8.0,51.0,2,1,0,2,3,0.01,5.0,1
3,8.0,28.0,2,0,3,3,3,0.01,5.0,1
4,14.0,67.0,2,0,3,3,3,0.01,7.0,0


In [13]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [14]:
def calc_vif(z):
    vif = pd.DataFrame()
    vif["variables"] = z.columns
    vif["VIF"] = [variance_inflation_factor(z.values, i) for i in range(z.shape[1])]
    return(vif)

In [15]:
z = dataset.iloc[:,0:9]
calc_vif(z)

,variables,VIF
0,DemAffl,6.278442
1,DemAge,10.735119
2,DemClusterGroup,3.659693
3,DemGender,1.435469
4,DemReg,2.474606
5,DemTVReg,3.752267
6,LoyalClass,3.851180
7,LoyalSpend,1.863146
8,LoyalTime,3.138948


from matplotlib import pyplot as plt
_df_0['VIF'].plot(kind='hist', bins=20, title='VIF')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['VIF'].plot(kind='line', figsize=(8, 4), title='VIF')
plt.gca().spines[['top', 'right']].set_visible(False)

In [16]:
y = dataset.iloc[:,9].values
x = dataset.iloc[:,0:9].values

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [18]:
classifier = LogisticRegression(max_iter=200)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
import joblib
joblib.dump(classifier, './c2_Classifier_LoyalCustomers')

['./c2_Classifier_LoyalCustomers']

In [20]:
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[3191  176]
 [ 689  389]]
0.8053993250843644


In [22]:
predictions = classifier.predict_proba(X_test)
predictions

array([[0.86223975, 0.13776025],
       [0.64513736, 0.35486264],
       [0.48564169, 0.51435831],
       ...,
       [0.88556565, 0.11443435],
       [0.90487362, 0.09512638],
       [0.87319097, 0.12680903]])

In [23]:
df_prediction_prob = pd.DataFrame(predictions, columns = ['prob0', 'prob1'])
df_test_dataset = pd.DataFrame(y_test, columns = ['Actual Outcome'])
df_x_test = pd.DataFrame(X_test)

dfx = pd.concat([df_x_test, df_test_dataset, df_prediction_prob], axis=1)
dfx.to_excel("Final_test_dataset.xlsx")
dfx.head()

,0,1,2,3,4,5,6,7,8,Actual Outcome,prob0,prob1
0,10.0,58.0,3.0,1.0,3.0,8.0,0.0,12632.66,4.0,0,0.862240,0.137760
1,10.0,51.0,2.0,0.0,3.0,8.0,0.0,6000.00,6.0,0,0.645137,0.354863
2,16.0,65.0,1.0,0.0,2.0,6.0,0.0,6053.35,7.0,0,0.485642,0.514358
3,5.0,60.0,3.0,0.0,3.0,3.0,2.0,5000.00,1.0,0,0.914295,0.085705
4,9.0,52.0,3.0,0.0,0.0,4.0,2.0,3500.00,6.0,0,0.702714,0.297286
